In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import yfinance as yf

In [ ]:
tickers = ["GOOGL", "NVDA", "META", "PLTR", "WFC",
           "DIS", "LUV", "PFE", "COKE", "CAT"]
market_ticker = "^GSPC"

data = yf.download(tickers + [market_ticker],
                   start="2022-01-01", end="2025-01-01")["Close"]

returns = np.log(data / data.shift(1)).dropna()

stock_returns = returns[tickers]
market_returns = returns[market_ticker]


In [ ]:
betas = []
idio_sigmas = []

for t in tickers:
    X = sm.add_constant(market_returns.values)
    y = stock_returns[t].values
    res = sm.OLS(y, X).fit()

    betas.append(res.params[1])
    idio_sigmas.append(res.resid.std())

betas = np.array(betas)
idio_sigmas = np.array(idio_sigmas)


In [ ]:
def get_weights(data):
    vol = data.std()
    return vol / vol.sum()

weights = get_weights(stock_returns).values


In [ ]:
n_steps = 252
n_paths = 1000

mu_m = market_returns.mean()
sigma_m = market_returns.std()

R_m = np.random.normal(mu_m, sigma_m, size=(n_steps, n_paths))

R_i = np.zeros((len(tickers), n_steps, n_paths))

for i in range(len(tickers)):
    eps = np.random.normal(0, idio_sigmas[i], size=(n_steps, n_paths))
    R_i[i] = betas[i] * R_m + eps

R_p = np.tensordot(weights, R_i, axes=(0, 0))


In [ ]:
alphas = np.zeros(n_paths)
betas_est = np.zeros(n_paths)

for k in range(n_paths):
    y = R_p[:, k]
    X = sm.add_constant(R_m[:, k])
    res = sm.OLS(y, X).fit()
    alphas[k] = res.params[0]
    betas_est[k] = res.params[1]


In [ ]:
plt.hist(alphas * 100, bins=50, density=True)
plt.title("Monte Carlo Alpha Distribution (%)")
plt.xlabel("Alpha (%)")
plt.show()

plt.hist(betas_est, bins=50, density=True)
plt.title("Monte Carlo Beta Distribution")
plt.xlabel("Beta")
plt.show()
